In [1]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.autograd import Variable

import torchvision

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [ ]:
# !unzip archive.zip -d data

In [2]:
model = torchvision.models.vgg19(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\apole/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
  3%|▎         | 15.4M/548M [00:10<06:02, 1.54MB/s]


KeyboardInterrupt: 

In [ ]:
for param in model.parameters():
    param.requires_grad = False
print(model)

In [ ]:
# Change the number of output features
model.classifier[6].out_features = 2

# Set requires_grad to True on the linear layer
for param in model.classifier.parameters():
    param.requires_grad = True

# Initialize the weights
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight.data)
        
model.classifier.apply(weights_init);

# Move the model to the GPU
model = model.cuda()

In [ ]:
data_transforms = {'train':
                    torchvision.transforms.Compose([
                    torchvision.transforms.RandomResizedCrop(224),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor()]),
                   'test':
                    torchvision.transforms.Compose([
                    torchvision.transforms.Resize(256),
                    torchvision.transforms.CenterCrop(224),
                    torchvision.transforms.ToTensor()])
                  }

In [ ]:
image_dataset = {x: torchvision.datasets.ImageFolder(os.path.join('./data/', x), data_transforms[x]) for x in ['train', 'test']}
data_loader = {x: DataLoader(image_dataset[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'test']}

In [ ]:
def imshow(imgs, title=None):
    """Imshow for Tensor."""
    imgs = imgs.numpy().transpose((1, 2, 0))
    plt.imshow(imgs)
    if title is not None:
        plt.title(title)
    


# Get a batch of training data
inputs, classes = next(iter(data_loader['train']))
class_names=['hot_dog', 'not_hot_dog']

# Make a grid from batch
imgs = torchvision.utils.make_grid(inputs)

imshow(imgs, title=[class_names[x] for x in classes])

In [ ]:
optimizer = optim.SGD(model.classifier.parameters(),lr=0.001, momentum=0.9, nesterov=True, weight_decay=1e-6)
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 300
dataset_sizes = {x: len(image_dataset[x]) for x in ['train', 'test']}

with tqdm_notebook(total=epochs,unit="epoch") as pbar:
    for epoch in range(epochs):
        running_loss = 0
        running_corrects = 0
        for i,data in enumerate(data_loader['train']):
            inputs, labels = data
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())


            optimizer.zero_grad()
            outputs = model(inputs)
            preds = torch.max(outputs.data, 1)[1]
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects / dataset_sizes['train']
        pbar.set_postfix(loss=epoch_loss, acc=epoch_acc)
        pbar.update()
        print(f'Epoch: {epoch}. Loss: {epoch_loss}. Train Acc: {epoch_acc}')